# Explainable AI Dashboard – Exploration Notebook

This notebook demonstrates the core XAI components independently,
outside the Streamlit dashboard, for experimentation and debugging.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

## 1. Load Data & Train Model

In [ ]:
df = pd.read_csv('../data/credit_risk_sample.csv')
print(f'Shape: {df.shape}')
df.head()

In [ ]:
target = 'default'
protected = ['age', 'gender']
feature_cols = [c for c in df.columns if c != target and c not in protected]

X = df[feature_cols].copy()
y = df[target].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
print(f'Accuracy: {model.score(X_test, y_test):.4f}')

## 2. SHAP – Global Explanations

In [ ]:
from src.explainability.shap_explainer import SHAPExplainer
from src.visualization.shap_plots import plot_global_importance, plot_beeswarm

shap_exp = SHAPExplainer(model, X_train.sample(100, random_state=42))
shap_vals = shap_exp.compute_shap_values(X_test)

importance = shap_exp.global_importance(X_test)
importance

In [ ]:
fig = plot_global_importance(shap_vals, list(X_test.columns))
fig.show()

In [ ]:
fig = plot_beeswarm(shap_vals, X_test, top_k=10)
fig.show()

## 3. LIME – Local Explanation

In [ ]:
from src.explainability.lime_explainer import LIMEExplainer
from src.visualization.lime_plots import plot_lime_explanation

lime_exp = LIMEExplainer(X_train, feature_names=list(X_train.columns))
result = lime_exp.explain_instance(model, X_test.iloc[0].values, num_features=10)

fig = plot_lime_explanation(result.feature_weights, result.prediction, result.prediction_proba)
fig.show()

## 4. Fairness Analysis

In [ ]:
from src.fairness.metrics import compute_all_metrics
from src.fairness.bias_detector import detect_bias, bias_summary_text

y_pred = model.predict(X_test)
sensitive = df.loc[X_test.index, 'gender'].values

bias_df = detect_bias(y_test.values, y_pred, sensitive, 'gender')
print(bias_summary_text(bias_df))
bias_df

In [ ]:
from src.visualization.fairness_plots import plot_bias_heatmap

fig = plot_bias_heatmap(bias_df)
fig.show()

## 5. Report Generation

In [ ]:
from src.models.metadata import extract_model_info, compute_performance
from src.reporting.summary_generator import generate_executive_summary, generate_technical_report, to_plain_text

info = extract_model_info(model, feature_names=list(X_train.columns))
perf = compute_performance(y_test, y_pred, model, X_test)

summary = generate_executive_summary(info, perf, bias_df, list(importance['feature'].head(5)))
print(summary)

In [ ]:
report = generate_technical_report(info, perf, fairness_results=bias_df)
print(to_plain_text(report))